# Spring 2021

## Raquel Aoki

Starting the project


Data simulation sources: 
- https://github.com/JiajingZ/CopulaSensitivity
- https://github.com/raquelaoki/ParKCa

text



In [1]:
import sys 
sys.path.insert(0,'src/')
from data_simulation import *

In [2]:
sim_data = copula_simulated_data(s = 10)
tr, u, y_continuous, y_binary = sim_data.get_data()
effect_true, effect_obs, true_treat_obs_effect_01,true_treat_obs_effect= sim_data.get_true_coefs()
sim_data.print_equation()


Outcomes (Y):
 [0, 0, 0, 0, 0] (binary) or  [  -1.43979215   -8.33416739 -182.1288151   -11.34852134 -190.67555226] (continuous)
Treatments: (8000, 4)
Confounders: (8000, 10)
True effect [0.73661721 0.73661721 1.26338279 0.98389964]
True obs effect [[1.01406203 0.63416235 1.3890912  1.02184143]]

Binary Nonlinear:
B: True treat. obs effect [[ 0.02752294 -0.74311927  0.79449541  0.04275229  0.        ]]

Continuous Nonlinear
C: True treat. obs effect [[ 0.02752294 -0.74311927  0.79449541  0.04275229]]

Equation:
g(T)=+3T0-1T1+1T2I(T2>0)+0.7T2I(T2<0)0T3-4T0*T0


In [3]:
#change the proportion of true causes
gwas_data = gwas_simulated_data(1000, 100, 8, prop_tc = 0.05)
y, tc, X = gwas_data.generate_samples()


In [4]:
y

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,